In [ ]:
from __future__ import unicode_literals, print_function, division

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import re
import os
import time
import copy
import unicodedata
import _pickle as cPickle
import random

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [ ]:
from google.colab import files
uploaded = files.upload()

Saving eng-fra.txt to eng-fra.txt


In [ ]:
REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')

def normalize_text(text):
    text = text.lower()
    text = REPLACE_BY_SPACE_RE.sub(' ',text)
    text = BAD_SYMBOLS_RE.sub('',text)
    
    return text


data_path = 'eng-fra.txt'

file = open(data_path,'r')
content = file.read()
og_pairs = [[normalize_text(s) for s in line.split('\t')] for line in content.split('\n')]

In [ ]:
print(pairs[:10])
print(len(pairs))

[['go', 'va '], ['run', 'cours'], ['run', 'courez'], ['wow', 'a alors'], ['fire', 'au feu '], ['help', ' laide'], ['jump', 'saute'], ['stop', 'a suffit'], ['stop', 'stop'], ['stop', 'arrtetoi ']]
135843


In [ ]:
MAX_len = 40
pairs = []
for pair in og_pairs:
  if len(pair) < 2:
    continue
  words_1 = pair[0].split(' ')
  words_2 = pair[1].split(' ')
  if len(words_1) < MAX_len and len(words_2) < MAX_len:
    pairs.append(pair)

In [ ]:
print(len(pairs))
print(pairs[-1])

135836
['a carbon footprint is the amount of carbon dioxide pollution that we produce as a result of our activities some people try to reduce their carbon footprint because they are concerned about climate change', 'une empreinte carbone est la somme de pollution au dioxyde de carbone que nous produisons par nos activits certaines personnes essaient de rduire leur empreinte carbone parce quelles sont inquites du changement climatique']


In [ ]:
class Language:
    
    def __init__(self,name):
        
        self.name = name
        self.num_words = 2
        self.word_to_index = {}
        self.index_to_word = {0: "SOS", 1: "EOS"}
        self.word_count = {}
        
    def add_word(self,word):
        
        if word not in self.word_to_index:
            
            self.word_to_index[word] = self.num_words
            self.word_count[word] = 1
            self.index_to_word[self.num_words] = word
            self.num_words += 1
        
        else:
            self.word_count[word] += 1

In [ ]:
eng = Language("eng")
fra = Language("fra")



for pair in pairs:
    
    if len(pair) < 2:
        continue

    for eng_word in pair[0].split(' '):
        eng.add_word(eng_word)
    for fra_word in pair[1].split(' '):
        fra.add_word(fra_word)
    
print(eng.num_words)
print(fra.num_words)

13662
26262


In [ ]:
print(eng.word_to_index)

{'go': 2, 'run': 3, 'wow': 4, 'fire': 5, 'help': 6, 'jump': 7, 'stop': 8, 'wait': 9, 'i': 10, 'see': 11, 'try': 12, 'won': 13, 'oh': 14, 'no': 15, 'attack': 16, 'cheers': 17, 'get': 18, 'up': 19, 'got': 20, 'it': 21, 'hop': 22, 'in': 23, 'hug': 24, 'me': 25, 'fell': 26, 'know': 27, 'left': 28, 'lost': 29, 'im': 30, '19': 31, 'ok': 32, 'listen': 33, 'way': 34, 'really': 35, 'thanks': 36, 'we': 37, 'ask': 38, 'tom': 39, 'awesome': 40, 'be': 41, 'calm': 42, 'cool': 43, 'fair': 44, 'kind': 45, 'nice': 46, 'beat': 47, 'call': 48, 'us': 49, 'come': 50, 'on': 51, 'drop': 52, 'out': 53, 'away': 54, 'slow': 55, 'goodbye': 56, 'hang': 57, 'he': 58, 'quit': 59, 'runs': 60, 'hold': 61, 'agree': 62, 'tried': 63, 'ill': 64, 'fat': 65, 'fit': 66, 'hit': 67, 'sad': 68, 'shy': 69, 'wet': 70, 'its': 71, 'join': 72, 'keep': 73, 'kiss': 74, '': 75, 'too': 76, 'open': 77, 'perfect': 78, 'you': 79, 'show': 80, 'shut': 81, 'so': 82, 'long': 83, 'take': 84, 'tell': 85, 'wake': 86, 'wash': 87, 'who': 88, 'am':

In [ ]:
eng_data = [eng.word_to_index,eng.index_to_word,eng.word_count,eng.num_words]

cPickle.dump( eng_data, open( "eng_dicts.p", "wb" ) )

fra_data = [fra.word_to_index,fra.index_to_word,fra.word_count,fra.num_words]

cPickle.dump( fra_data, open( "fra_dicts.p", "wb" ) )

In [ ]:
SOS_token = 0
EOS_token = 1

def tensor_from_sentence(sentence,lang):
    
    if lang == 'eng':
      indices = [eng.word_to_index[word] for word in sentence.split(' ')]
    elif lang == 'fra':
      indices = [fra.word_to_index[word] for word in sentence.split(' ')]
    indices.append(EOS_token)
    
    return torch.tensor(indices, dtype=torch.long, device=device).view(-1, 1)

def tensors_from_pair(pair):

    input_tensor = tensor_from_sentence( pair[0],'eng')
    target_tensor = tensor_from_sentence(pair[1],'fra')
    
    return (input_tensor, target_tensor)

In [ ]:
class EncoderRNN(nn.Module):
    
    def __init__(self, input_size, hidden_size):
        
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)

    def forward(self, input, hidden):
        
        embedded = self.embedding(input).view(1, 1, -1)
        output = embedded
        output, hidden = self.gru(output, hidden)
        
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

In [ ]:
class AttnDecoderRNN(nn.Module):
    
    def __init__(self, hidden_size, output_size, dropout_p=0.1, max_length=45):
        
        super(AttnDecoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.dropout_p = dropout_p
        self.max_length = max_length

        self.embedding = nn.Embedding(self.output_size, self.hidden_size)
        self.attn = nn.Linear(self.hidden_size * 2, self.max_length)
        self.attn_combine = nn.Linear(self.hidden_size * 2, self.hidden_size)
        self.dropout = nn.Dropout(self.dropout_p)
        self.gru = nn.GRU(self.hidden_size, self.hidden_size)
        self.out = nn.Linear(self.hidden_size, self.output_size)

    def forward(self, input, hidden, encoder_outputs):
        
        embedded = self.embedding(input).view(1, 1, -1)
        embedded = self.dropout(embedded)

        attn_weights = F.softmax(
            self.attn(torch.cat((embedded[0], hidden[0]), 1)), dim=1)
        attn_applied = torch.bmm(attn_weights.unsqueeze(0),
                                 encoder_outputs.unsqueeze(0))

        output = torch.cat((embedded[0], attn_applied[0]), 1)
        output = self.attn_combine(output).unsqueeze(0)

        output = F.relu(output)
        output, hidden = self.gru(output, hidden)

        output = F.log_softmax(self.out(output[0]), dim=1)
        
        return output, hidden, attn_weights

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

In [ ]:
teacher_forcing_ratio = 0.5


def train(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion, max_length=45):
    
    encoder_hidden = encoder.initHidden()

    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    input_length = input_tensor.size(0)
    target_length = target_tensor.size(0)

    encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

    loss = 0

    for ei in range(input_length):
        encoder_output, encoder_hidden = encoder(
            input_tensor[ei], encoder_hidden)
        encoder_outputs[ei] = encoder_output[0, 0]

    decoder_input = torch.tensor([[SOS_token]], device=device)

    decoder_hidden = encoder_hidden

    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False

    if use_teacher_forcing:
        # Teacher forcing: Feed the target as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            loss += criterion(decoder_output, target_tensor[di])
            decoder_input = target_tensor[di]  # Teacher forcing

    else:
        # Without teacher forcing: use its own predictions as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            topv, topi = decoder_output.topk(1)
            decoder_input = topi.squeeze().detach()  # detach from history as input

            loss += criterion(decoder_output, target_tensor[di])
            if decoder_input.item() == EOS_token:
                break

    loss.backward()

    encoder_optimizer.step()
    decoder_optimizer.step()

    return loss.item() / target_length

In [ ]:
import matplotlib.pyplot as plt
plt.switch_backend('agg')
import matplotlib.ticker as ticker
import numpy as np


def showPlot(points):
    plt.figure()
    fig, ax = plt.subplots()
    loc = ticker.MultipleLocator(base=0.2)
    ax.yaxis.set_major_locator(loc)
    plt.plot(points)

In [ ]:
import time
import math


def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)


def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))

In [ ]:
def trainIters(encoder, decoder, n_iters, print_every=1000, plot_every=100, learning_rate=0.01):
    start = time.time()
    plot_losses = []
    print_loss_total = 0  # Reset every print_every
    plot_loss_total = 0  # Reset every plot_every

    encoder_optimizer = optim.SGD(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.SGD(decoder.parameters(), lr=learning_rate)
    training_pairs = [tensors_from_pair(random.choice(pairs))
                      for i in range(n_iters)]
    criterion = nn.NLLLoss()

    for iter in range(1, n_iters + 1):
        training_pair = training_pairs[iter - 1]
        input_tensor = training_pair[0]
        target_tensor = training_pair[1]

        loss = train(input_tensor, target_tensor, encoder,
                     decoder, encoder_optimizer, decoder_optimizer, criterion)
        print_loss_total += loss
        plot_loss_total += loss

        if iter % print_every == 0:
            print_loss_avg = print_loss_total / print_every
            print_loss_total = 0
            print('%s (%d %d%%) %.4f' % (timeSince(start, iter / n_iters),
                                         iter, iter / n_iters * 100, print_loss_avg))

        if iter % plot_every == 0:
            plot_loss_avg = plot_loss_total / plot_every
            plot_losses.append(plot_loss_avg)
            plot_loss_total = 0

    showPlot(plot_losses)

In [ ]:
def evaluate(encoder, decoder, sentence, max_length=45):
    
    with torch.no_grad():
        input_tensor = tensor_from_sentence(sentence,'eng')
        input_length = input_tensor.size()[0]
        encoder_hidden = encoder.initHidden()

        encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

        for ei in range(input_length):
            
            encoder_output, encoder_hidden = encoder(input_tensor[ei],
                                                     encoder_hidden)
            encoder_outputs[ei] += encoder_output[0, 0]

        decoder_input = torch.tensor([[SOS_token]], device=device)  # SOS

        decoder_hidden = encoder_hidden

        decoded_words = []
        decoder_attentions = torch.zeros(max_length, max_length)

        for di in range(max_length):
            
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            decoder_attentions[di] = decoder_attention.data
            topv, topi = decoder_output.data.topk(1)
            
            if topi.item() == EOS_token:
                decoded_words.append('<EOS>')
                break
            else:
                decoded_words.append(fra.index_to_word[topi.item()])

            decoder_input = topi.squeeze().detach()

        return decoded_words, decoder_attentions[:di + 1]

In [ ]:
def evaluateRandomly(encoder, decoder, n=10):
    
    for i in range(n):
        pair = random.choice(pairs)
        print('>', pair[0])
        print('=', pair[1])
        output_words, attentions = evaluate(encoder, decoder, pair[0])
        output_sentence = ' '.join(output_words)
        print('<', output_sentence)
        print('')

In [ ]:
hidden_size = 256

encoder1 = EncoderRNN(eng.num_words, hidden_size).to(device)
attn_decoder1 = AttnDecoderRNN(hidden_size, fra.num_words, dropout_p=0.1).to(device)

trainIters(encoder1, attn_decoder1, 75000, print_every=5000)

3m 46s (- 52m 50s) (5000 6%) 5.2816
7m 32s (- 48m 58s) (10000 13%) 4.8319
11m 24s (- 45m 37s) (15000 20%) 4.5520
15m 21s (- 42m 13s) (20000 26%) 4.3207
19m 15s (- 38m 31s) (25000 33%) 4.1458
23m 13s (- 34m 50s) (30000 40%) 4.0111
27m 11s (- 31m 4s) (35000 46%) 3.9347
31m 12s (- 27m 18s) (40000 53%) 3.7765
35m 12s (- 23m 28s) (45000 60%) 3.7069
39m 14s (- 19m 37s) (50000 66%) 3.6147
43m 19s (- 15m 45s) (55000 73%) 3.5711
47m 18s (- 11m 49s) (60000 80%) 3.5027
51m 17s (- 7m 53s) (65000 86%) 3.4293
55m 16s (- 3m 56s) (70000 93%) 3.3823
59m 19s (- 0m 0s) (75000 100%) 3.3810


In [ ]:
evaluateRandomly(encoder1, attn_decoder1)

> dont sneak out of the concert
= ne file pas du concert
< ne laissez pas de la <EOS>

> tom tried to keep calm
= tom essaya de rester calme
< tom a essay de de <EOS>

> it was a good day
= a a t une bonne journe
< ctait un bon jour <EOS>

> he was so busy that he sent his son instead of going himself
= il tait tellement occup quil envoya son fils  sa place
< il tait tellement de son de son pre de son pre <EOS>

> dont tell me what to say
= ne me dicte pas ce que je dois dire
< ne dites pas dire  dire  <EOS>

> youre very brave
= tu es fort brave
< vous tes trs gentil <EOS>

> i bought two pairs of pants
= jai achet deux pantalons
< jai achet deux deux de deux <EOS>

> we intended to start right away
= on avait lintention de commencer de suite
< nous avons demand de de <EOS>

> ive been busy all week
= jai t occup toute la semaine
< jai t tout ce <EOS>

> above all  logic requires precise definitions
= surtout  la logique ncessite des dfinitions prcises
< tout  a a   <EOS>



In [ ]:
print("encoder model: \n\n", encoder1, '\n')
print("The state dict keys: \n\n", encoder1.state_dict().keys())
print(" ")
print("attn_decoder model: \n\n", attn_decoder1, '\n')
print("The state dict keys: \n\n", attn_decoder1.state_dict().keys())

encoder model: 

 EncoderRNN(
  (embedding): Embedding(13662, 256)
  (gru): GRU(256, 256)
) 

The state dict keys: 

 odict_keys(['embedding.weight', 'gru.weight_ih_l0', 'gru.weight_hh_l0', 'gru.bias_ih_l0', 'gru.bias_hh_l0'])
 
attn_decoder model: 

 AttnDecoderRNN(
  (embedding): Embedding(26262, 256)
  (attn): Linear(in_features=512, out_features=45, bias=True)
  (attn_combine): Linear(in_features=512, out_features=256, bias=True)
  (dropout): Dropout(p=0.1, inplace=False)
  (gru): GRU(256, 256)
  (out): Linear(in_features=256, out_features=26262, bias=True)
) 

The state dict keys: 

 odict_keys(['embedding.weight', 'attn.weight', 'attn.bias', 'attn_combine.weight', 'attn_combine.bias', 'gru.weight_ih_l0', 'gru.weight_hh_l0', 'gru.bias_ih_l0', 'gru.bias_hh_l0', 'out.weight', 'out.bias'])


In [ ]:
torch.save(encoder1.state_dict(), 'checkpoint_enc.pth')
files.download('checkpoint_enc.pth')

torch.save(attn_decoder1.state_dict(),'checkpoint_dec.pth')
files.download('checkpoint_dec.pth')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>